In [1]:
import torch
from IPython import display
from d2l import torch as d2l

batch_siz=256
train_iter,test_iter =d2l.load_data_fashion_mnist(batch_siz)

In [2]:
num_inputs=784
num_outputs=10

In [3]:
W=torch.normal(0,0.01,size=(num_inputs,num_outputs),requires_grad=True)
b=torch.zeros(num_outputs,requires_grad=True)

In [4]:
def softmax(X):
    X_exp=torch.exp(X)
    partition=X_exp.sum(1,keepdim=True)
    return X_exp/partition

In [ ]:
 #实现 softmax 回归
def net(X):
    return softmax(torch.matmul(X.reshape((-1,W.shape[0])),W)+b)